In [126]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from rfpimp import permutation_importances
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [127]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

In [128]:
# In order to get reproducible results

# Seed value (can actually be different for each attribution step)
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

In [129]:
def load_data():
    
    def dummie_and_drop(df, name):
        # Creates a dummy variable, concatenates it and finally drops the original categorical variable.
        # In order not to have redundant variables, one of the dummy variables is dropped too
        dummies = pd.get_dummies(df[name]).rename(columns = lambda x: name + '_' + str(x))
        dummies = dummies.drop(dummies.columns[-1], axis = 1)
        df = pd.concat([df, dummies], axis = 1)
        df.drop(columns = [name], inplace=True, axis=1)

        return df
    
    def convert_to_categorical(df, categorical_variables, categories, need_pickup = True):
        """ 
        The dataframe's selected variables are converted to categorical, and each variable's categories are also specified.
        It is also specified if the "pickup community area" has to be converted into categorical or no. If it is not 
        converted into categorical it is because it's not going to be used in the model.            
        """
        
        if need_pickup:
            begin = 0
        else:
            df.drop(columns = ['pickup_community_area'], inplace = True, axis = 1)
            begin = 1
        
        for i in range(begin, len(categorical_variables)):
            df[categorical_variables[i]] = df[categorical_variables[i]].astype('category').cat.set_categories(categories[i])
        return df
    
    
    def load(name, need_pickup = False, drop_correlated = False):
    
        # This parameter has to be set to True if the "pickup_community_area" variable is needed in the model
        

        # Load needed dataset and choose the useful columns
        df = pd.read_csv(name) #'dataset_train.csv')

        x = df[['pickup_community_area' ,'temperature', 'relative_humidity', 'wind_direction', 'wind_speed', 'precipitation_cat', 
                'sky_level', 'daytype', 'Day Name', 'Month', 'Hour', 'Fare Last Month', 'Trips Last Hour',
                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)', 'Year']]
#        float32=['temperature','relative_humidity','wind_direction','wind_speed','Fare Last Month', 'Trips Last Hour',
#                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)']
#        x= x[float32]=x[float32].astype('float32')
        # Convert the categorical variables
        categorical_variables = ['pickup_community_area', 'daytype', 'sky_level', 'Day Name', 'Month','Hour', 'Year']
        categories = [[*(range(1,78))], ['U', 'W', 'A'], ['OVC', 'BKN', 'SCT', 'FEW', 'CLR', 'VV '], 
                      ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
                      [*(range(1,13))], [*(range(0, 24))], ['2017', '2018', '2019']]

        
        
        x = convert_to_categorical(x, categorical_variables, categories, need_pickup = need_pickup)

        float32=['temperature','relative_humidity','wind_direction','wind_speed','Fare Last Month', 'Trips Last Hour',
                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)']
        
        x[float32]=x[float32].astype('float32')
        # Make dummy variables with the categorical ones
        if need_pickup:
            begin = 0
        else:
            begin = 1
        for i in range(begin, len(categorical_variables)):
            x = dummie_and_drop(x, name = categorical_variables[i])

        y = df['Trips'].to_numpy()

        if need_pickup == False:
            # If we don't need the pickup, it means this is Neural Network case. Therefore we have to modify Y, in order
            # to have "n_areas" outputs per input (because there are "n_areas" regressions per input)
            n_areas = 77
            y = np.reshape(y, [-1, n_areas]) # If 
        
        if drop_correlated:
            x.drop(columns = ['Trips Last Week (Same Hour)'], inplace = True, axis = 1)
            x.drop(columns = ['Trips 2 Weeks Ago (Same Hour)'], inplace = True, axis = 1)

#        x = x.to_numpy()
        
        return (x,y)   
    
# ------------------------------------- MAIN PROGRAM ------------------------

    need_pickup = True 
    drop_correlated = False
    
    
    name_train = 'dataset_train.csv'
#    name_test = 'dataset_test.csv'
    x, y = load(name_train, need_pickup, drop_correlated)
#    x_test, y_test = load(name_test, need_pickup, drop_correlated)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15,shuffle=True)
    
    return (x_train, x_test, y_train, y_test)

In [130]:
x_train, x_test, y_train, y_test=load_data()

C:\Users\Iconsense\Anaconda3\envs\abhishek\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [131]:
x_train.shape

(1584937, 134)

In [132]:
rf1 = RandomForestRegressor(n_estimators = 128,bootstrap=True,min_samples_leaf=8,oob_score=True,n_jobs=-1,max_features=0.5,verbose=4)

In [ ]:
rf1=rf1.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 128
building tree 2 of 128
building tree 3 of 128
building tree 4 of 128
building tree 5 of 128
building tree 6 of 128
building tree 7 of 128
building tree 8 of 128
building tree 9 of 128
building tree 10 of 128
building tree 11 of 128
building tree 12 of 128
building tree 13 of 128
building tree 14 of 128
building tree 15 of 128building tree 16 of 128

building tree 17 of 128
building tree 18 of 128
building tree 19 of 128
building tree 20 of 128
building tree 21 of 128
building tree 22 of 128
building tree 23 of 128
building tree 24 of 128
building tree 25 of 128
building tree 26 of 128
building tree 27 of 128
building tree 28 of 128
building tree 29 of 128
building tree 30 of 128
building tree 31 of 128
building tree 32 of 128
building tree 33 of 128
building tree 34 of 128
building tree 35 of 128
building tree 36 of 128
building tree 37 of 128
building tree 38 of 128
building tree 39 of 128
building tree 40 of 128
building tree 41 of 128
building tree 42 of 128
b

[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:  3.9min


building tree 78 of 128
building tree 79 of 128
building tree 80 of 128
building tree 81 of 128
building tree 82 of 128
building tree 83 of 128
building tree 84 of 128
building tree 85 of 128
building tree 86 of 128
building tree 87 of 128
building tree 88 of 128
building tree 89 of 128
building tree 90 of 128
building tree 91 of 128
building tree 92 of 128
building tree 93 of 128
building tree 94 of 128
building tree 95 of 128
building tree 96 of 128
building tree 97 of 128
building tree 98 of 128
building tree 99 of 128
building tree 100 of 128
building tree 101 of 128
building tree 102 of 128
building tree 103 of 128
building tree 104 of 128
building tree 105 of 128
building tree 106 of 128
building tree 107 of 128
building tree 108 of 128
building tree 109 of 128
building tree 110 of 128
building tree 111 of 128
building tree 112 of 128
building tree 113 of 128
building tree 114 of 128
building tree 115 of 128
building tree 116 of 128
building tree 117 of 128
building tree 118 of 1

[Parallel(n_jobs=-1)]: Done 122 out of 128 | elapsed:  8.2min remaining:   24.0s
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:  8.2min finished


In [ ]:
#rf1.feature_importances_

In [ ]:
def metric(rf):
    print("Traning Score")
    print(rf.score(x_train,y_train))
    print("Test Score")
    print("MAE Train")
    print(mean_absolute_error(np.rint(rf.predict(x_train), y_train)))
    print("MAE Test")
    print(mean_absolute_error(np.rint(rf.predict(x_train), y_test)))

    print("MSE Train")
    print(mean_squared_error(np.rint(rf.predict(x_train)), y_train))
    print("MSE Test")
    print(mean_squared_error(np.rint(rf.predict(x_test)), y_test))
    print("MAPE Train")
    print(mean_absolute_percentage_error(y_test,np.rint(rf6.predict(x_test))))
    print("MAPE Test")
    print(mean_absolute_percentage_error(y_test,np.rint(rf6.predict(x_test))))

In [ ]:
metric(rf1)

In [ ]:
def feature_Imp(rf):
    importances = rf1.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(x_train.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    
    print("Top 10 Feature")
    print(x_train.columns[indices[:10]])
    l=len(indices)
    print("Least Important 10 Feature")
    print(x_train.columns[indices[(l-10):]])

In [ ]:
# with open('rf1.pickle', 'wb') as f:
#     pickle.dump(rf1, f)

In [ ]:
# with open('rf1.pickle', 'rb') as f:
        
#         rf1 = pickle.load(f)

In [ ]:
def load_data():
    
    def dummie_and_drop(df, name):
        # Creates a dummy variable, concatenates it and finally drops the original categorical variable.
        # In order not to have redundant variables, one of the dummy variables is dropped too
        dummies = pd.get_dummies(df[name]).rename(columns = lambda x: name + '_' + str(x))
        dummies = dummies.drop(dummies.columns[-1], axis = 1)
        df = pd.concat([df, dummies], axis = 1)
        df.drop(columns = [name], inplace=True, axis=1)

        return df
    
    def convert_to_categorical(df, categorical_variables, categories, need_pickup = True):
        """ 
        The dataframe's selected variables are converted to categorical, and each variable's categories are also specified.
        It is also specified if the "pickup community area" has to be converted into categorical or no. If it is not 
        converted into categorical it is because it's not going to be used in the model.            
        """
        
        if need_pickup:
            begin = 0
        else:
            df.drop(columns = ['pickup_community_area'], inplace = True, axis = 1)
            begin = 1
        
        for i in range(begin, len(categorical_variables)):
            df[categorical_variables[i]] = df[categorical_variables[i]].astype('category').cat.set_categories(categories[i])
        return df
    
    
    def load(name, need_pickup = False, drop_correlated = False):
    
        # This parameter has to be set to True if the "pickup_community_area" variable is needed in the model
        

        # Load needed dataset and choose the useful columns
        df = pd.read_csv(name) #'dataset_train.csv')

        x = df[['pickup_community_area' ,'temperature', 'relative_humidity', 'wind_direction', 'wind_speed', 'precipitation_cat', 
                'sky_level', 'daytype', 'Day Name', 'Month', 'Hour', 'Fare Last Month', 'Trips Last Hour',
                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)', 'Year']]
#        float32=['temperature','relative_humidity','wind_direction','wind_speed','Fare Last Month', 'Trips Last Hour',
#                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)']
#        x= x[float32]=x[float32].astype('float32')
        # Convert the categorical variables
        categorical_variables = ['pickup_community_area', 'daytype', 'sky_level', 'Day Name', 'Month','Hour', 'Year']
        categories = [[*(range(1,78))], ['U', 'W', 'A'], ['OVC', 'BKN', 'SCT', 'FEW', 'CLR', 'VV '], 
                      ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
                      [*(range(1,13))], [*(range(0, 24))], ['2017', '2018', '2019']]

        
        
        x = convert_to_categorical(x, categorical_variables, categories, need_pickup = need_pickup)

        float32=['temperature','relative_humidity','wind_direction','wind_speed','Fare Last Month', 'Trips Last Hour',
                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)']
        
        x[float32]=x[float32].astype('float32')
        # Make dummy variables with the categorical ones
        if need_pickup:
            begin = 0
        else:
            begin = 1
        for i in range(begin, len(categorical_variables)):
            x = dummie_and_drop(x, name = categorical_variables[i])

        y = df['Trips'].to_numpy()

        if need_pickup == False:
            # If we don't need the pickup, it means this is Neural Network case. Therefore we have to modify Y, in order
            # to have "n_areas" outputs per input (because there are "n_areas" regressions per input)
            n_areas = 77
            y = np.reshape(y, [-1, n_areas]) # If 
        
        if drop_correlated:
            x.drop(columns = ['Trips Last Week (Same Hour)'], inplace = True, axis = 1)
            x.drop(columns = ['Trips 2 Weeks Ago (Same Hour)'], inplace = True, axis = 1)

#        x = x.to_numpy()
        
        return (x,y)   
    
# ------------------------------------- MAIN PROGRAM ------------------------

    need_pickup = True 
    drop_correlated = True
    
    
    name_train = 'dataset_train.csv'
#    name_test = 'dataset_test.csv'
    x, y = load(name_train, need_pickup, drop_correlated)
#    x_test, y_test = load(name_test, need_pickup, drop_correlated)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15,shuffle=True)
    
    return (x_train, x_test, y_train, y_test)

In [ ]:
x_train, x_test, y_train, y_test=load_data()

In [ ]:
rf2 = RandomForestRegressor(n_estimators = 128,bootstrap=True,min_samples_leaf=8,oob_score=True,n_jobs=-1,max_features=0.5,verbose=4)

rf2=rf2.fit(x_train, y_train)

rf2.score(x_train,y_train)

In [ ]:
metric(rf2)

In [ ]:
def load_data():
    
    def dummie_and_drop(df, name):
        # Creates a dummy variable, concatenates it and finally drops the original categorical variable.
        # In order not to have redundant variables, one of the dummy variables is dropped too
        dummies = pd.get_dummies(df[name]).rename(columns = lambda x: name + '_' + str(x))
        dummies = dummies.drop(dummies.columns[-1], axis = 1)
        df = pd.concat([df, dummies], axis = 1)
        df.drop(columns = [name], inplace=True, axis=1)

        return df
    
    def convert_to_categorical(df, categorical_variables, categories, need_pickup = True):
        """ 
        The dataframe's selected variables are converted to categorical, and each variable's categories are also specified.
        It is also specified if the "pickup community area" has to be converted into categorical or no. If it is not 
        converted into categorical it is because it's not going to be used in the model.            
        """
        
        if need_pickup:
            begin = 0
        else:
            df.drop(columns = ['pickup_community_area'], inplace = True, axis = 1)
            begin = 1
        
        for i in range(begin, len(categorical_variables)):
            df[categorical_variables[i]] = df[categorical_variables[i]].astype('category').cat.set_categories(categories[i])
        return df
    
    
    def load(name, need_pickup = False, drop_correlated = False):
    
        # This parameter has to be set to True if the "pickup_community_area" variable is needed in the model
        

        # Load needed dataset and choose the useful columns
        df = pd.read_csv(name) #'dataset_train.csv')

        x = df[['pickup_community_area' ,'temperature', 'relative_humidity', 'wind_direction', 'wind_speed', 'precipitation_cat', 
                'sky_level', 'daytype', 'Day Name', 'Month', 'Hour', 'Fare Last Month', 'Trips Last Hour',
                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)', 'Year']]
#        float32=['temperature','relative_humidity','wind_direction','wind_speed','Fare Last Month', 'Trips Last Hour',
#                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)']
#        x= x[float32]=x[float32].astype('float32')
        # Convert the categorical variables
        categorical_variables = ['pickup_community_area', 'daytype', 'sky_level', 'Day Name', 'Month','Hour', 'Year']
        categories = [[*(range(1,78))], ['U', 'W', 'A'], ['OVC', 'BKN', 'SCT', 'FEW', 'CLR', 'VV '], 
                      ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
                      [*(range(1,13))], [*(range(0, 24))], ['2017', '2018', '2019']]

        
        
        x = convert_to_categorical(x, categorical_variables, categories, need_pickup = need_pickup)

        float32=['temperature','relative_humidity','wind_direction','wind_speed','Fare Last Month', 'Trips Last Hour',
                'Trips Last Week (Same Hour)', 'Trips 2 Weeks Ago (Same Hour)']
        
        x[float32]=x[float32].astype('float32')
        # Make dummy variables with the categorical ones
        if need_pickup:
            begin = 0
        else:
            begin = 1
        for i in range(begin, len(categorical_variables)):
            x = dummie_and_drop(x, name = categorical_variables[i])

        y = df['Trips'].to_numpy()

        if need_pickup == False:
            # If we don't need the pickup, it means this is Neural Network case. Therefore we have to modify Y, in order
            # to have "n_areas" outputs per input (because there are "n_areas" regressions per input)
            n_areas = 77
            y = np.reshape(y, [-1, n_areas]) # If 
        
        if drop_correlated:
            x.drop(columns = ['Trips Last Week (Same Hour)'], inplace = True, axis = 1)
            x.drop(columns = ['Trips 2 Weeks Ago (Same Hour)'], inplace = True, axis = 1)

#        x = x.to_numpy()
        
        return (x,y)   
    
# ------------------------------------- MAIN PROGRAM ------------------------

    need_pickup = True 
    drop_correlated = False
    
    
    name_train = 'dataset_train.csv'
#    name_test = 'dataset_test.csv'
    x, y = load(name_train, need_pickup, drop_correlated)
#    x_test, y_test = load(name_test, need_pickup, drop_correlated)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15,shuffle=True)
    
    return (x_train, x_test, y_train, y_test)

In [ ]:
x_train, x_test, y_train, y_test=load_data()

In [ ]:
rf3 = RandomForestRegressor(n_estimators = 256,bootstrap=True,min_samples_leaf=64,oob_score=True,n_jobs=-1,
                           max_features='sqrt',verbose=4,min_samples_split=64 )

rf3=rf3.fit(x_train, y_train)


In [ ]:
metric(rf3)

In [ ]:
rf4 = RandomForestRegressor(n_estimators = 256,bootstrap=True,min_samples_leaf=64,oob_score=True,n_jobs=-1,
                           max_features=0.4,verbose=4,min_samples_split=64 )
rf4=rf4.fit(x_train, y_train)


In [ ]:
metric(rf4)

In [ ]:
rf5 = RandomForestRegressor(n_estimators =128 ,bootstrap=True,min_samples_leaf=32,oob_score=True,n_jobs=-1,max_features=0.6,verbose=4)

rf5=rf5.fit(x_train, y_train)

In [ ]:
metric(rf5)

In [ ]:
rf6 = RandomForestRegressor(n_estimators =128 ,bootstrap=True,min_samples_leaf=32,oob_score=True,n_jobs=-1,max_features='sqrt',verbose=4)

rf6=rf6.fit(x_train, y_train)


In [ ]:
metric(rf6)

In [ ]:
# with open('rf6.pickle', 'wb') as f:
#     pickle.dump(rf6, f)



In [ ]:
# with open('rf6.pickle', 'rb') as f:
        
#         rf6 = pickle.load(f)